In [1]:
# Импорт библиотек
import datetime
import requests
import json
from pybit import spot
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from pybit.exceptions import InvalidRequestError
import numpy as np
from settings import *
from Def import *
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime,timedelta
import signal
import time



In [2]:
#Для страты
top_adx = 0
lowest_adx = 0
top_DI_pos= 0

#параметры запроса
symbol = 'BTCUSD'
interval = '15'
fields = "open_time,open,high,low,close,volume"

#создаем пустой DataFrame
df = get_prev_data()
try:
    while True:
    # получаем текущее время на сервере Bybit
        url = 'https://api.bybit.com/v2/public/time'
        response = requests.get(url)
        server_time = json.loads(response.text)['time_now']

        # преобразуем server_time в целочисленный тип
        server_time = int(float(server_time))

        # вычисляем время начала текущего бара
        start_time = int(time.time()) - 900

        # ожидаем ближайшей секунды текущей минуты
        current_time = datetime.fromtimestamp(server_time).strftime('%S.%f')
        wait_time = 899.0 - float(current_time)
        time.sleep(wait_time)

        # Запрос 
        url = f"https://api.bybit.com/v2/public/kline/list?symbol={symbol}&interval={interval}&from={start_time}&limit=200&fields={fields}"
        response = requests.get(url, headers={"api-key": API, "sign": secret})
        data = json.loads(response.text)
        new_df = pd.DataFrame(data['result'])

        # преобразование времени в удобный для чтения формат
        new_df['open_time'] = pd.to_datetime(new_df['open_time'], unit='s')

        # добавляем новые данные в DataFrame
        new_data = new_df[~new_df['open_time'].isin(df['open_time'])]
        df = pd.concat([df, new_data], ignore_index=True)

        # удаляем колонку turnover
        df = df.drop(columns=["turnover","interval"])

        # Применение страты
        if not new_data.empty:
            df.to_csv('data.csv', index=False)
            dfn = pd.read_csv('data.csv')

            # Вычисляем ADX, DI+ и DI-
            dfn['ADX'], dfn['DI+'], dfn['DI-'] = adx(dfn['high'], dfn['low'], dfn['close'], n=14)

            # Создаем столбцы с предыдущими значениями ADX, DI+ и DI-, low
            dfn['prev_adx'] = dfn['ADX'].shift()
            dfn['prev_di_plus'] = dfn['DI+'].shift()
            dfn['prev2_di_minus'] = dfn['DI-'].shift()
            dfn['prev_low'] = dfn['low'].shift()
            last_adx = dfn['ADX'].iloc[-1]
            prev2_adx = dfn['ADX'].iloc[-3]
            back_adx = dfn['ADX'].iloc[-2]
            diff_adx = back_adx - last_adx
            top_adx = 0
            lowest_adx = 0
            top_DI_pos= 0

            # Устанавливаем начальное значение флага позиции
            position = 'out'

            # Создаем список для хранения сигналов
            signals = []
            
            for index, row in dfn.iterrows():
                if row['ADX']>20 and row['ADX']>row['prev_adx']:
                    top_adx=row['ADX']
                elif row['ADX']<20:
                    top_adx=0
                
                # Условия покупки
                if position == 'out':
                    if row['DI+'] - row['DI-'] > 15 and row['DI+'] > row['prev_di_plus'] and row['ADX']>20 and row['diff_adx']>2:
                        signals.append('buy')
                        print('Buy:',row['close'])
                        position = 'long'
                    else:
                        signals.append('hold')

                # Условия продажи
                elif position == 'long':
                    if top_adx - row['ADX']>15:
                        signals.append('sell')
                        print('Sell:',row['close'])
                        position = 'out'
                    elif row['DI-'] > row['DI+']:
                        signals.append('sell')
                        print('Sell:',row['close'])
                        position = 'out'
                    else:
                        signals.append('hold')

            dfn.to_csv('dfn.csv', index=False)
            
except KeyboardInterrupt:
    print("Программа остановлена пользователем")